# Segement Vs Kinesis Comparision 

In [8]:
# config values..
event_log_s3_path = "s3://ms-data-pipeline-production/ms-data-stream-production-processed"
event_log_s3_bucket = event_log_s3_path.split("s3://")[1].split("/")[0]
event_log_s3_prefix = event_log_s3_path.split("/")[-1]
athena_bucket_path = "s3://aws-athena-query-results-358002497134-ap-southeast-1/"
#athena_database = "ms_data_processed_production"
athena_database = "data_science"
athena_raw_events_table = "ms_data_stream_production_processed"
#athena_raw_events_table = "ms_data_stream_production_processed_cd5f4696237059a21d780afa83822e6b"
athena_year_partition = "partition_0" #2019
athena_month_partition = "partition_1" #02
athena_day_partition = "partition_2" #0
athena_easy_events_table =  "id_ab_test"
aws_region_name = "ap-southeast-1"


In [9]:
import sys
try:
    from pyathena import connect
except:
    print("Failed to import pyathena, trying to install it")
    !{sys.executable} -m pip install PyAthena
    from pyathena import connect
    print("successfully installed")
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
    
import pandas as pd
import athenaConnection

# Events from Athena (KINESIS)

In [10]:
print("Connecting to Athena")
athena = athenaConnection.AthenaQuery()
athena_conn, athena_cursor =athena.connect_to_athena()
athena.connect()


Connecting to Athena


In [11]:
print("Running small test query to check the athena connection")
query = "select * from data_science.ms_data_stream_production_processed limit 10"
#pd.read_sql(query, athena_conn)#using pandas
kdf=athena.query(query)#using custom implementation

Running small test query to check the athena connection


In [12]:
kdf.columns

Index(['type', 'context', 'user', 'body', 'sent_at', 'partition_0',
       'partition_1', 'partition_2'],
      dtype='object')

# Events from Postges (SEGMENT)

In [14]:
import pgdbConnection
pgConn=pgdbConnection.get_connection()

Connecting to database
  ->host=data-pier-production.cl8qfdl47mtr.ap-southeast-1.rds.amazonaws.com port=5432 dbname=data_pier user=pari password=ja5DAmTK8hD3zuPpnNaySJJcgYxnLGJD
Connected!



In [15]:
sdf = pd.read_sql("SELECT * FROM moneysmartsg_prod.lead_generation_conversion LIMIT 2",con=pgConn)

In [17]:
sdf.columns

Index(['id', 'received_at', 'context_campaign_source',
       'context_library_version', 'is_paid', 'list_position', 'page_referrer',
       'channel', 'context_page_url', 'destination_url', 'event_type',
       'eventtype', 'product_name', 'form_step', 'is_sponsored', 'page_path',
       'sent_at', 'anonymous_id', 'context_user_agent', 'event_text',
       'productname', 'provider', 'context_ip', 'context_page_referrer',
       'product_id', 'productid', 'context_campaign_medium',
       'context_campaign_name', 'context_library_name', 'timestamp', 'uuid_ts',
       'auth_status', 'context_page_path', 'context_page_search',
       'context_page_title', 'product_category', 'event', 'is_recommended',
       'original_timestamp', 'user_id', 'action', 'pr_4_4_s_novider',
       'object_clicked', 'product_variant_id', 'cta_type', 'provider_id'],
      dtype='object')

# Join segment & kinesis

In [24]:
kdf_events=athena.query("SELECT body.event_name as page_view_event,user.anonymous_id,context.page_url,sent_at FROM data_science.ms_data_stream_production_processed WHERE body.event_name='PageView' AND context.page_url LIKE '%moneysmart.sg%' AND sent_at>= '2019-12-10'")

In [55]:
kdf_events.columns

Index(['page_view_event', 'anonymous_id', 'page_url', 'sent_at'], dtype='object')

In [22]:
sdf_events=pd.read_sql("SELECT anonymous_id,context_page_url,received_at from moneysmartsg_prod.pages WHERE CAST(received_at AS DATE) >= '2019-12-10'",con=pgConn)

In [54]:
sdf_events.columns

Index(['anonymous_id', 'context_page_url', 'received_at'], dtype='object')

In [44]:
merged_inner = pd.merge(kdf_events,sdf_events, on=['anonymous_id','anonymous_id'])

In [60]:
merged_inner.shape

(2419420, 6)

In [59]:
merged_inner[(merged_inner['anonymous_id'] == merged_inner['anonymous_id']) ].shape

(2419420, 6)